# References: 
* [P4 Multispectral Image Processing Guide](https://dl.djicdn.com/downloads/p4-multispectral/20200717/P4_Multispectral_Image_Processing_Guide_EN.pdf) 
* [MicaSense RedEdge and Altum Image Processing Tutorials](https://github.com/micasense/imageprocessing)

In [1]:
### IMPORT LIBRARIES

import os, glob, shutil
import cv2
import numpy as np
import pandas as pd
from osgeo import gdal
import helper.metadata as metadata

import exiftool
exiftoolPath = None
if os.name == 'nt':
    exiftoolPath = 'C:/exiftool/exiftool(-k).exe'
with exiftool.ExifTool(exiftoolPath) as exift:
    print('Exiftool works!')
    
print ('required libraries are imported...')

Exiftool works!
required libraries are imported...


In [2]:
### SET PARAMETERS

SvyFolder = r"C:\UAV\Projects\DjiTest" # <----- EDIT THIS
NumBands = 5 # Blue, Green, Red, RedEdge, NIR
ksize, n_iter, eps = 5, 2500, 1e-9 # Image alignment parameters
Overwrite = False

ReflectanceImages =  SvyFolder + '\\Reflectance'
StackedImages =  SvyFolder + '\\Stacked'
UnstackedImages =  SvyFolder + '\\Unstacked'

if Overwrite and os.path.exists(StackedImages):
    shutil.rmtree(StackedImages)
if not os.path.exists(StackedImages):
    os.mkdir(StackedImages)

if os.path.exists(UnstackedImages):
    shutil.rmtree(UnstackedImages)
os.mkdir(UnstackedImages)

In [ ]:
### ALIGNING AND STACKING

file_set, band_set, img_set, aligned_set = [], [], [], []
no_error = True
df = pd.DataFrame(columns=["SourceFile",
                           "FileName",
                           "Make",
                           "Model",
                           "RelativeAltitude",
                           "FocalLength",
                           "GPSAltitudeRef",
                           "GPSLatitudeRef",
                           "GPSLongitudeRef",
                           "GPSAltitude",
                           "GPSLatitude",
                           "GPSLongitude"])

files = glob.glob(ReflectanceImages + "\*.tif")
n_files = len(files)
print("Total number of images:", n_files)
n_sets = int(len(files) / NumBands)
print("Total number of image sets:", n_sets)

meta_csv = pd.read_csv(os.path.join(ReflectanceImages, "meta.csv")).set_index("FileName")


for i in range(n_sets):
    print("Processing {} / {} image sets".format(i+1, n_sets))
    
    file_name = files[0].split("\\")[-1]
    meta = meta_csv.loc[file_name]
    file_id = file_name.split("DJI_")[1].split(".TIF")[0]
    set_id = str(file_id[0:-len(str(NumBands))])
    band_id = str(file_id[-len(str(NumBands))])
    
    # Get all bands from the same set
    for j in range(1,NumBands+1):
        
        file_name = "DJI_" + set_id + str(j) + ".TIF"
        file_path = os.path.join(ReflectanceImages, file_name)
        image = cv2.imread(file_path, cv2.COLOR_BGR2GRAY)
        meta = meta_csv.loc[file_name]
        band = meta["BandName"]        
        files.remove(file_path)
        
        # Save set
        file_set.append(file_name)
        band_set.append(band)
        img_set.append(image)
        if band == "NIR":
            df_nir = {
            "SourceFile": os.path.join(StackedImages, "DJI_SET{}.TIF".format(set_id)),
            "FileName": "DJI_SET{}.TIF".format(set_id),
            "Make": meta["Make"],
            "Model": meta["Model"],
            "RelativeAltitude": meta["RelativeAltitude"],
            "FocalLength": meta["FocalLength"],
            "GPSAltitudeRef": meta["GPSAltitudeRef"],
            "GPSLatitudeRef": meta["GPSLatitudeRef"],
            "GPSLongitudeRef": meta["GPSLongitudeRef"],
            "GPSAltitude": meta["GPSAltitude"],
            "GPSLatitude": meta["GPSLatitude"],
            "GPSLongitude": meta["GPSLongitude"]
            }
            df = df.append(df_nir, ignore_index = True)
            
    
    # ----- Align difference caused by different exposure times -----
    # Reference: https://learnopencv.com/image-alignment-ecc-in-opencv-c-python/
    
    def get_gradient(im) :
        # Calculate the x and y gradients using Sobel operator
        grad_x = cv2.Sobel(im,cv2.CV_32F,1,0,ksize=ksize)
        grad_y = cv2.Sobel(im,cv2.CV_32F,0,1,ksize=ksize)

        # Combine the two gradients
        grad = cv2.addWeighted(np.absolute(grad_x), 0.5, np.absolute(grad_y), 0.5, 0)
        return grad
    
    if Overwrite or not os.path.exists(os.path.join(StackedImages, "DJI_SET{}.TIF".format(set_id))):
        
        # Find NIR band
        NIR_id = int(band_set.index("NIR"))
        file_NIR = file_set[NIR_id]

        # Define motion model
        warp_mode = cv2.MOTION_HOMOGRAPHY

        # Set the warp matrix to identity
        if warp_mode == cv2.MOTION_HOMOGRAPHY :
            warp_matrix = np.eye(3, 3, dtype=np.float32)
        else :
            warp_matrix = np.eye(2, 3, dtype=np.float32)

        # Set the stopping criteria for the algorithm
        criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, n_iter,  eps)

        # Warp the other channels to the NIR channel
        for k in range(len(img_set)):  
            image_NIR = np.array(img_set[NIR_id])
            image_k = np.array(img_set[k])
            height_k, width_k = image_k.shape[:2]
            if k != NIR_id:
                try:
                    (cc, warp_matrix) = cv2.findTransformECC(get_gradient(image_NIR), 
                                                             get_gradient(image_k),
                                                             warp_matrix, 
                                                             warp_mode, 
                                                             criteria,
                                                             inputMask=None, 
                                                             gaussFiltSize=1)

                    # Use Perspective warp when the transformation is a Homography
                    if warp_mode == cv2.MOTION_HOMOGRAPHY :
                        i_warp = cv2.warpPerspective (image_k,
                                                      warp_matrix, 
                                                      (width_k,height_k),
                                                      flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)

                    # Use Affine warp when the transformation is not a Homography
                    else :
                        i_warp = cv2.warpAffine(image_k, 
                                                warp_matrix, 
                                                (width_k, height_k), 
                                                flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)

                    aligned_set.append(i_warp)

                except:
                    no_error = False
                    print("ERROR: {} could not be aligned".format(file_set[k]))
                    cv2.imwrite(os.path.join(UnstackedImages, file_set[k]), image_k)

            else:
                aligned_set.append(image_NIR)
                
            # Export raster
            if no_error:
                height, width = aligned_set[0].shape[:2]
                driver = gdal.GetDriverByName('GTiff')
                outRaster = driver.Create(os.path.join(StackedImages, "DJI_SET{}.TIF".format(set_id)), 
                                          width, height, NumBands, 
                                          gdal.GDT_Float32, 
                                          options = ['INTERLEAVE=BAND', 'COMPRESS=DEFLATE'])

                for n in range(len(aligned_set)):
                    outband = outRaster.GetRasterBand(n+1)
                    outband.WriteArray(aligned_set[n])
                    outband.FlushCache()
                outRaster = None

    file_set, band_set, img_set, aligned_set = [], [], [], []
    no_error = True

print("Completed!")

Total number of images: 1315
Total number of image sets: 263
Processing 1 / 263 image sets


In [ ]:
### TRANSFER METADATA

df.to_csv(os.path.join(StackedImages, "meta.csv"), index=False)

cwd = os.getcwd()
os.chdir(StackedImages)
!exiftool -csv=meta.csv -overwrite_original {StackedImages}
os.chdir(cwd)
print("Completed!")